In [1]:
import json
import casanova
from collections import Counter

In [2]:
reader = casanova.reader("../../data/toflit18_all_flows.csv")

#Filters
year_col = reader.headers["year"]
dirferme_col = reader.headers["customs_region"]
exchange_col = reader.headers["export_import"]
source_col = reader.headers["best_guess_region_prodxpart"]

#Fields
partner_col = reader.headers["partner_grouping"]
value_col = reader.headers["value"]

imports = Counter()
exports = Counter()
for row in reader:
    if row[year_col] == "1789" and row[dirferme_col] == "Marseille" and row[source_col] == "1":
        if row[exchange_col] == "Imports":
            imports[row[partner_col]] += float(row[value_col] or 0)
        elif row[exchange_col] == "Exports":
            exports[row[partner_col]] += float(row[value_col] or 0)

In [3]:
imports

Counter({'Nord': 3283456.5936220004,
         'Hollande': 2046345.67102488,
         'Italie': 43052914.571335405,
         'Amériques': 31740648.0,
         'Levant et Barbarie': 39436933.45469722,
         'Espagne': 8840868.18039737,
         'Portugal': 304088.001869,
         'France': 32135889.123546023,
         "États-Unis d'Amérique": 958093.7048460001,
         "Flandre et autres états de l'Empereur": 823740.383224,
         'Afrique': 10756.0,
         'Angleterre': 1046333.0,
         '[vide]': 2201.0})

In [4]:
exports

Counter({'France': 36427133.12979482,
         'Monde': 24912626.0,
         'Angleterre': 1140719.80401517,
         'Italie': 13691097.147279572,
         'Allemagne': 2052.0,
         'Levant et Barbarie': 13304798.760940531,
         'Espagne': 9863096.925390964,
         'Nord': 1178146.32251836,
         "Flandre et autres états de l'Empereur": 282884.47705189005,
         'Hollande': 895695.7541407199,
         "États-Unis d'Amérique": 239788.98914249995,
         'Amériques': 14542393.0,
         'Afrique': 408552.0,
         '????': 360.0,
         'Asie': 1660684.0})

In [5]:
partners = set([k for k in imports.keys()] + [k for k in exports.keys()])

In [6]:
with open("partner_grouping_latlong.json") as f:
    coords = json.load(f)

geoloc = lambda place: {
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": coords.get(place)
  },
  "properties": {
    "name": place,
    "imports": imports.get(place),
    "exports": exports.get(place)
  }
}

geojson_data = {
    "type": "FeatureCollection",
    "features": [geoloc(p) for p in partners if p in coords]
}

In [7]:
geojson_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [8.2397, 39.6797]},
   'properties': {'name': 'Portugal',
    'imports': 304088.001869,
    'exports': None}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-15.414, 13.454]},
   'properties': {'name': 'Afrique', 'imports': 10756.0, 'exports': 408552.0}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [4.2764, 50.8242]},
   'properties': {'name': "Flandre et autres états de l'Empereur",
    'imports': 823740.383224,
    'exports': 282884.47705189005}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [5.427, 40.271]},
   'properties': {'name': 'Monde', 'imports': None, 'exports': 24912626.0}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [4.9878, 52.2984]},
   'properties': {'name': 'Hollande',
    'imports': 2046345.67102488,
    'exports': 895695.7541407199}},
  {'type': 'Feature',

In [9]:
with open("map/marseille_partner_groupings_import_export_geoloc.json", "w") as f:
    json.dump(geojson_data, f)